In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)

import nltk
import string 
nltk.download('punkt')
from nltk.corpus import stopwords

import ast
from bert_serving.client import BertClient

import torch
import torch.nn as nn
import torch.optim as optim

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
excel_data =  pd.read_csv("sentiment.csv")
del excel_data['Unnamed: 0']

excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000130712.jpg'].index , inplace=True)
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000310622.jpg'].index , inplace=True)
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000421673.jpg'].index , inplace=True)
excel_data.drop(excel_data[excel_data.filename == 'COCO_val2014_000000359276.jpg'].index , inplace=True)

data = excel_data[['imgid', 'filename', 'successful', 'tokens', 'word_sentiment', 'sentiment']]

In [3]:
tokens =  data['tokens']
sen = data['word_sentiment']

# converting String tokens to actual list

In [4]:
converted_tokens = []
for token in tokens:
    string = token
    sets = ast.literal_eval(string)
    converted_tokens.append(sets)

In [5]:
converted_sentiments = []
for sentiment in sen:
    new_sen = [float(x.strip()) for x in sentiment.strip('[]').split(',')]
    converted_sentiments.append(new_sen)

In [6]:
data['tokens'] = converted_tokens
data['word_sentiment'] = converted_sentiments

C:\Users\user\AppData\Local\Temp\ipykernel_1888\3501353506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokens'] = converted_tokens
C:\Users\user\AppData\Local\Temp\ipykernel_1888\3501353506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['word_sentiment'] = converted_sentiments


we are going to send final sentiments to tell these strings havve overall positive/negitive sentiment and now the modedl will have to find which words are cotribuiting to that sentiment

# convertinng to 5 grams 

includes the middle token, 2 before and after middle token

In [7]:
def convert(token, sentiment):
    if (len(token)%2 == 1):
        middle_index = len(token) // 2
    else:
        middle_index = (len(token) // 2 )- 1

    gram = token[middle_index-2:middle_index+3]
    sentiment = sentiment[middle_index-2:middle_index+3]
    
    return (gram, sentiment)

In [8]:
import copy
datacopy = copy.deepcopy(data)
data = datacopy

In [9]:
gram5_tokens = []
gram5_sentiments = []
target_token = []
target_sentiment = []

for index, row in data.iterrows():
    tokens = row[3]; sent = row[4]
    
    gram5 = convert(tokens, sent)
    
    gram5_tokens.append(gram5[0])
    gram5_sentiments.append(gram5[1])
    
    target_token.append(gram5[0][2])
    target_sentiment.append(gram5[1][2])

data['tokens'] = gram5_tokens
data['word_sentiment'] = gram5_sentiments
data['target_token'] = target_token
data['target_sentiment'] = target_sentiment

In [10]:
data = data[["tokens" , "word_sentiment" , 'target_token', 'target_sentiment', "sentiment"]]

In [11]:
data['text'] = data['tokens'].apply(lambda x: ' '.join(x))

C:\Users\user\AppData\Local\Temp\ipykernel_1888\1873234889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['tokens'].apply(lambda x: ' '.join(x))


# Feature Eextraction

In [12]:
text= np.array(data["text"])
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
vectorizer = TfidfVectorizer()
vectorizer.fit(text.astype('U'))
testing=vectorizer.transform(text.astype('U'))
matrix=testing.todense()

In [14]:
df= pd.DataFrame(testing.toarray(), columns= vectorizer.get_feature_names())
df.head()

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,555,abandoned,able,aboard,about,above,absolutely,abstract,abused,accented,...,you,young,your,zebra,zebras,zone,zoo,zookeeper,zooming,zooms
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


features to tensor

In [15]:
import torch
def convert_to_tensor(file):
    tensor = torch.tensor(file, dtype=torch.float32)
    return tensor

In [16]:
text_tensor = convert_to_tensor(matrix)

#to check if all tensors have same lenght
leng = []
for i in text_tensor:
    leng.append(len(i))
    
set(leng)

{3105}

dump features using pickle

In [17]:
import pickle
# Write the dictionary to a file using pickle.dump
with open('task2.pkl', 'wb') as f:
    pickle.dump(text_tensor, f)

# define model

In [36]:
class MyModel_Multi(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(in_features=3105, out_features=2500)
        self.l2 = nn.Linear(in_features=2500, out_features=2000)
        self.l3 = nn.Linear(in_features=2000, out_features=1000)
        self.l4 = nn.Linear(in_features=4096, out_features=2500)
        self.l5 = nn.Linear(in_features=2500, out_features=2000)
        self.l6 = nn.Linear(in_features=2000, out_features=1000)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x1):
        x1 = self.l1(x1)
        x1 = self.sigmoid(x1)
        x1 = self.l2(x1)
        x1 = self.sigmoid(x1)
        x1 = self.l3(x1)
        x1 = self.sigmoid(x1)
        x1 = self.l4(x1)
        x1 = self.sigmoid(x1)
        x1 = self.l5(x1)
        x1 = self.sigmoid(x1)
        x1 = self.l6(x1)
        x1 = self.sigmoid(x1)

        return x1

In [51]:
class sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(in_features=3105, out_features=2500)
        self.bn1 = nn.BatchNorm1d(num_features=2500)
        self.l2 = nn.Linear(in_features=2500, out_features=2000)
        self.bn2 = nn.BatchNorm1d(num_features=2000)
        self.l3 = nn.Linear(in_features=2000, out_features=1000)
        self.bn3 = nn.BatchNorm1d(num_features=1000)
        self.l4 = nn.Linear(in_features=1000, out_features=500)
        self.bn4 = nn.BatchNorm1d(num_features=500)
        self.l5 = nn.Linear(in_features=500, out_features=250)
        self.bn5 = nn.BatchNorm1d(num_features=250)
        self.l6 = nn.Linear(in_features=250, out_features=100)
        self.bn6 = nn.BatchNorm1d(num_features=100)
        self.l7 = nn.Linear(in_features=100, out_features=50)
        self.bn7 = nn.BatchNorm1d(num_features=50)
        self.l8 = nn.Linear(in_features=50, out_features=1)

        self.dropout = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l5(x)
        x = self.bn5(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l6(x)
        x = self.bn6(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l7(x)
        x = self.bn7(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.l8(x)
        x = self.sigmoid(x)

        return x

In [82]:
class sentiment_sigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(in_features=3105, out_features=2500)
        self.l2 = nn.Linear(in_features=2500, out_features=2000)
        self.l3 = nn.Linear(in_features=2000, out_features=1000)
        self.l4 = nn.Linear(in_features=1000, out_features=500)
        self.l5 = nn.Linear(in_features=500, out_features=250)
        self.l6 = nn.Linear(in_features=250, out_features=100)
        self.l7 = nn.Linear(in_features=100, out_features=50)
        self.l8 = nn.Linear(in_features=50, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.sigmoid(x)

        x = self.l2(x)
        x = self.sigmoid(x)

        x = self.l3(x)
        x = self.sigmoid(x)

        x = self.l4(x)
        x = self.sigmoid(x)

        x = self.l5(x)
        x = self.sigmoid(x)

        x = self.l6(x)
        x = self.sigmoid(x)

        x = self.l7(x)
        x = self.sigmoid(x)

        x = self.l8(x)
        x = self.sigmoid(x)

        return x

In [83]:
model = sentiment_sigmoid()
# model  = MyModel_Multi()
loss = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Load features

In [21]:
with open('task2.pkl', 'rb') as f:
    text = pickle.load(f)

# target sentiment to tensor

In [73]:
sentiment = data['target_sentiment'] # assuming 'sentiment' is a pandas series object
sentiment_array = np.array([sentiment.to_numpy()]).T
target = torch.tensor(sentiment_array, dtype=torch.float32)

In [74]:
with open('task2.pkl', 'rb') as f:
    text_tensors = pickle.load(f)

In [75]:
print(text_tensors.shape, text_tensors.dtype)

torch.Size([39109, 3105]) torch.float32


In [76]:
def split(train_size = 0.8):
    if train_size > 1:
        print("sending default 0.8 train size")
        ret = split(0.8)
        return ret
    else:
        rows = int(len(text_tensors)*train_size)
        data = [ text_tensors[:rows, :], target[:rows, :], text_tensors[rows:, :], target[rows:, :] ]
        return data

In [77]:
xtrain, ytrain, xtest, ytest= split(0.8)

In [78]:
ytrain

tensor([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]])

In [79]:
outputs

tensor([[0.4577],
        [0.4577],
        [0.4577],
        ...,
        [0.4577],
        [0.4577],
        [0.4577]], grad_fn=<SigmoidBackward0>)

# Train Model

In [84]:
# Train the model
for epoch in range(1):
    
    # Forward pass
    outputs = model(xtrain)
    loss = loss(outputs, ytrain)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print progress
    if (epoch+1) % 1 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 3, loss.item()))

Epoch [1/3], Loss: 0.9364


# save model

In [86]:
# assume 'model' is your trained PyTorch model
torch.save(model.state_dict(), 'task2_model.pth')

In [87]:
# create a new instance of your model
model2 = sentiment_sigmoid()
# load the saved parameters into the model
model2.load_state_dict(torch.load('task2_model.pth'))

<All keys matched successfully>

# Evaluation

In [88]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [89]:
model2.eval()

sentiment_sigmoid(
  (l1): Linear(in_features=3105, out_features=2500, bias=True)
  (l2): Linear(in_features=2500, out_features=2000, bias=True)
  (l3): Linear(in_features=2000, out_features=1000, bias=True)
  (l4): Linear(in_features=1000, out_features=500, bias=True)
  (l5): Linear(in_features=500, out_features=250, bias=True)
  (l6): Linear(in_features=250, out_features=100, bias=True)
  (l7): Linear(in_features=100, out_features=50, bias=True)
  (l8): Linear(in_features=50, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)